# Tier B: The Semanticist - AI vs Human Text Detection
## Using Averaged Pre-trained Word Embeddings (GloVe) + Feedforward NN

This notebook implements a binary classifier that distinguishes AI-generated from human-written text using:
- **Pre-trained GloVe embeddings** (glove.6B.100d)
- **Averaged embedding vectors** for each paragraph
- **Feedforward Neural Network** (PyTorch)

**Author**: Tier B Implementation  
**Dataset**: Human novels (class1) + AI-generated paragraphs (class2)  
**Model**: Feedforward NN with averaged embeddings

---
## 1. Environment Setup & Imports

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import json
import re
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                              f1_score, roc_auc_score, confusion_matrix, 
                              classification_report)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

print("✓ All imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

---
## 2. Data Preparation

**Data Sources:**
- **Class 1 (Human)**: Cleaned novel texts from 5 novels, chunked into ~200-word paragraphs
- **Class 2 (AI-generated)**: Pre-generated paragraphs (500 per novel = 2500 total)

**Process:**
1. Load cleaned human text and chunk into paragraphs
2. Load AI-generated JSONL files
3. Combine into a single dataset with labels (0=Human, 1=AI)
4. Create a DataFrame and save as CSV

In [ ]:
# Configure paths for Kaggle vs Local execution
# In Kaggle, you'll upload the output/ folder as a dataset
# For local testing, use relative paths

import os

# Detect if running in Kaggle
IN_KAGGLE = os.path.exists('/kaggle/input')

if IN_KAGGLE:
    # Kaggle paths - adjust based on your uploaded dataset name
    BASE_PATH = Path('/kaggle/input/precog-novels-data')  # Change this to your dataset name
    CLASS1_PATH = BASE_PATH / 'class1'
    CLASS2_PATH = BASE_PATH / 'class2'
else:
    # Local paths
    BASE_PATH = Path('../output')
    CLASS1_PATH = BASE_PATH / 'class1'
    CLASS2_PATH = BASE_PATH / 'class2'

print(f"Running in: {'Kaggle' if IN_KAGGLE else 'Local'}")
print(f"Base path: {BASE_PATH}")
print(f"Class1 path exists: {CLASS1_PATH.exists()}")
print(f"Class2 path exists: {CLASS2_PATH.exists()}")

In [ ]:
def chunk_text(text, chunk_size=200):
    """
    Chunk text into paragraphs of approximately chunk_size words.
    
    Args:
        text: Input text string
        chunk_size: Target number of words per chunk
    
    Returns:
        List of text chunks
    """
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.split()) >= 50:  # Minimum 50 words per chunk
            chunks.append(chunk)
    
    return chunks


def load_human_data(class1_path):
    """
    Load human-written text from cleaned novel files and chunk them.
    
    Returns:
        List of dictionaries with 'text' and 'label' keys
    """
    novels = [
        'heart_of_darkness_cleaned.txt',
        'lord_jim_cleaned.txt',
        'metamorphosis_cleaned.txt',
        'the_trial_cleaned.txt',
        'typhoon_cleaned.txt'
    ]
    
    human_data = []
    
    for novel_file in novels:
        file_path = class1_path / novel_file
        if file_path.exists():
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
            
            # Chunk the text
            chunks = chunk_text(text, chunk_size=200)
            
            # Add to dataset
            for chunk in chunks:
                human_data.append({
                    'text': chunk,
                    'label': 0,  # 0 = Human
                    'source': novel_file.replace('_cleaned.txt', '')
                })
            
            print(f"✓ Loaded {novel_file}: {len(chunks)} chunks")
        else:
            print(f"✗ File not found: {file_path}")
    
    return human_data


def load_ai_data(class2_path):
    """
    Load AI-generated text from JSONL files.
    
    Returns:
        List of dictionaries with 'text' and 'label' keys
    """
    novels = [
        'heart_of_darkness_generic.jsonl',
        'lord_jim_generic.jsonl',
        'metamorphosis_generic.jsonl',
        'the_trial_generic.jsonl',
        'typhoon_generic.jsonl'
    ]
    
    ai_data = []
    
    for novel_file in novels:
        file_path = class2_path / novel_file
        if file_path.exists():
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
            
            # Parse JSONL
            for line in lines:
                try:
                    entry = json.loads(line.strip())
                    # Extract text (adjust key based on your JSONL structure)
                    text = entry.get('text') or entry.get('paragraph') or entry.get('content', '')
                    
                    if text and len(text.split()) >= 50:  # Minimum 50 words
                        ai_data.append({
                            'text': text,
                            'label': 1,  # 1 = AI
                            'source': novel_file.replace('_generic.jsonl', '')
                        })
                except json.JSONDecodeError:
                    continue
            
            print(f"✓ Loaded {novel_file}: {len([d for d in ai_data if novel_file.replace('_generic.jsonl', '') in d['source']])} paragraphs")
        else:
            print(f"✗ File not found: {file_path}")
    
    return ai_data


# Load all data
print("Loading Human data (Class 1)...")
human_data = load_human_data(CLASS1_PATH)

print("\nLoading AI data (Class 2)...")
ai_data = load_ai_data(CLASS2_PATH)

# Combine datasets
all_data = human_data + ai_data

print(f"\n{'='*60}")
print(f"Total Human paragraphs: {len(human_data)}")
print(f"Total AI paragraphs: {len(ai_data)}")
print(f"Total dataset size: {len(all_data)}")
print(f"{'='*60}")

In [ ]:
# Create DataFrame
df = pd.DataFrame(all_data)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display dataset info
print("Dataset Overview:")
print(df.head())
print(f"\nDataset shape: {df.shape}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nSource distribution:")
print(df['source'].value_counts())

In [ ]:
# Perform 80/20 train-test split (stratified)
train_df, test_df = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df['label'], 
    random_state=42
)

print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"\nTraining set label distribution:")
print(train_df['label'].value_counts())
print(f"\nTest set label distribution:")
print(test_df['label'].value_counts())

---
## 3. Load Pre-trained GloVe Embeddings

**GloVe (Global Vectors for Word Representation)**
- Using `glove.6B.100d` (100-dimensional embeddings trained on 6B tokens)
- In Kaggle, GloVe embeddings are available as a dataset
- We'll load them into a dictionary: `{word: vector}`

In [ ]:
def load_glove_embeddings(glove_path, embedding_dim=100):
    """
    Load GloVe embeddings from file.
    
    Args:
        glove_path: Path to GloVe file
        embedding_dim: Dimension of embeddings (100, 200, 300, etc.)
    
    Returns:
        Dictionary mapping words to embedding vectors
    """
    embeddings = {}
    
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    
    return embeddings


# Configure GloVe path
if IN_KAGGLE:
    # In Kaggle, add GloVe as a dataset: https://www.kaggle.com/datasets/watts2/glove6b50dtxt
    GLOVE_PATH = '/kaggle/input/glove6b100dtxt/glove.6B.100d.txt'
else:
    # For local testing, download from: https://nlp.stanford.edu/projects/glove/
    # Place it in a known location
    GLOVE_PATH = 'glove.6B.100d.txt'  # Update this path

# Load embeddings
print("Loading GloVe embeddings...")
print(f"Path: {GLOVE_PATH}")

try:
    embeddings_dict = load_glove_embeddings(GLOVE_PATH, embedding_dim=100)
    EMBEDDING_DIM = 100
    print(f"✓ Loaded {len(embeddings_dict)} word embeddings")
    print(f"Embedding dimension: {EMBEDDING_DIM}")
    
    # Show sample embeddings
    sample_words = ['the', 'intelligence', 'artificial', 'human', 'text']
    print("\nSample embeddings:")
    for word in sample_words:
        if word in embeddings_dict:
            print(f"  {word}: {embeddings_dict[word][:5]}... (showing first 5 dims)")
        else:
            print(f"  {word}: NOT FOUND")
            
except FileNotFoundError:
    print("⚠ GloVe file not found!")
    print("In Kaggle: Add 'GloVe' dataset to your notebook")
    print("Locally: Download from https://nlp.stanford.edu/projects/glove/")

---
## 4. Text to Vector Conversion

Convert each paragraph into a fixed-size vector by:
1. Tokenizing the text (lowercase, whitespace split)
2. Looking up each word in the GloVe embeddings
3. Averaging all word vectors
4. Handling out-of-vocabulary (OOV) words gracefully

In [ ]:
def tokenize(text):
    """
    Simple tokenization: lowercase and split by whitespace.
    Remove punctuation and special characters.
    """
    # Lowercase
    text = text.lower()
    # Remove punctuation (keep only alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Split into words
    words = text.split()
    return words


def text_to_vector(text, embeddings_dict, embedding_dim):
    """
    Convert text to a single vector by averaging word embeddings.
    
    Args:
        text: Input text string
        embeddings_dict: Dictionary of word embeddings
        embedding_dim: Dimension of embeddings
    
    Returns:
        numpy array of shape (embedding_dim,)
    """
    words = tokenize(text)
    
    # Collect embeddings for known words
    word_vectors = []
    for word in words:
        if word in embeddings_dict:
            word_vectors.append(embeddings_dict[word])
    
    # Average the vectors
    if len(word_vectors) > 0:
        avg_vector = np.mean(word_vectors, axis=0)
    else:
        # No known words - return zero vector
        avg_vector = np.zeros(embedding_dim)
    
    return avg_vector


# Convert all texts to vectors
print("Converting texts to vectors...")

X_train_vecs = np.array([
    text_to_vector(text, embeddings_dict, EMBEDDING_DIM) 
    for text in train_df['text'].values
])

X_test_vecs = np.array([
    text_to_vector(text, embeddings_dict, EMBEDDING_DIM) 
    for text in test_df['text'].values
])

y_train = train_df['label'].values
y_test = test_df['label'].values

print(f"✓ Training vectors shape: {X_train_vecs.shape}")
print(f"✓ Test vectors shape: {X_test_vecs.shape}")
print(f"✓ Training labels shape: {y_train.shape}")
print(f"✓ Test labels shape: {y_test.shape}")

# Check for any zero vectors (completely OOV paragraphs)
train_zero_vecs = np.sum(np.all(X_train_vecs == 0, axis=1))
test_zero_vecs = np.sum(np.all(X_test_vecs == 0, axis=1))
print(f"\nZero vectors in training set: {train_zero_vecs}")
print(f"Zero vectors in test set: {test_zero_vecs}")

---
## 5. PyTorch Dataset Preparation

Create PyTorch Dataset and DataLoader for efficient batch processing during training.

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_vecs)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test_vecs)
y_test_tensor = torch.FloatTensor(y_test)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    drop_last=False
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    drop_last=False
)

print(f"✓ Training batches: {len(train_loader)}")
print(f"✓ Test batches: {len(test_loader)}")
print(f"✓ Batch size: {BATCH_SIZE}")

---
## 6. Feedforward Neural Network Model

**Architecture:**
- Input layer: 100 dimensions (GloVe embedding size)
- Hidden layer 1: 64 neurons + ReLU + Dropout(0.3)
- Hidden layer 2: 32 neurons + ReLU + Dropout(0.3)
- Output layer: 1 neuron + Sigmoid (binary classification)

In [ ]:
class FeedforwardClassifier(nn.Module):
    """
    Simple Feedforward Neural Network for binary classification.
    Uses averaged word embeddings as input.
    """
    
    def __init__(self, input_dim, hidden_dim1=64, hidden_dim2=32, dropout_rate=0.3):
        super(FeedforwardClassifier, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)
        
        self.fc3 = nn.Linear(hidden_dim2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        
        x = self.fc3(x)
        x = self.sigmoid(x)
        
        return x


# Initialize model
model = FeedforwardClassifier(
    input_dim=EMBEDDING_DIM,
    hidden_dim1=64,
    hidden_dim2=32,
    dropout_rate=0.3
).to(device)

# Print model architecture
print("Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

---
## 7. Training

**Training Configuration:**
- Loss function: Binary Cross Entropy (BCE)
- Optimizer: Adam
- Learning rate: 0.001
- Epochs: 20

In [ ]:
# Training configuration
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NUM_EPOCHS = 20

# Training history
train_losses = []
train_accuracies = []

print("Starting training...\n")
print("=" * 70)

for epoch in range(NUM_EPOCHS):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track metrics
        epoch_loss += loss.item()
        predictions = (outputs > 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)
    
    # Calculate epoch metrics
    avg_loss = epoch_loss / len(train_loader)
    accuracy = 100 * correct / total
    
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    
    print(f"Epoch [{epoch+1:2d}/{NUM_EPOCHS}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.2f}%")

print("=" * 70)
print("✓ Training complete!")

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss curve
ax1.plot(range(1, NUM_EPOCHS + 1), train_losses, marker='o', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss over Epochs')
ax1.grid(True, alpha=0.3)

# Accuracy curve
ax2.plot(range(1, NUM_EPOCHS + 1), train_accuracies, marker='o', linewidth=2, color='green')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training Accuracy over Epochs')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 8. Evaluation

Evaluate the model on the test set and compute comprehensive metrics.

In [ ]:
# Evaluation function
def evaluate_model(model, data_loader, device):
    """
    Evaluate the model and return predictions and probabilities.
    """
    model.eval()
    all_predictions = []
    all_probabilities = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs).squeeze()
            probabilities = outputs.cpu().numpy()
            predictions = (outputs > 0.5).float().cpu().numpy()
            
            all_predictions.extend(predictions)
            all_probabilities.extend(probabilities)
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_labels), np.array(all_predictions), np.array(all_probabilities)


# Get predictions on test set
y_true, y_pred, y_prob = evaluate_model(model, test_loader, device)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob)
cm = confusion_matrix(y_true, y_pred)

# Print results
print("=" * 70)
print("TEST SET EVALUATION RESULTS")
print("=" * 70)
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")
print("=" * 70)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Human', 'AI'], digits=4))

In [ ]:
# Confusion Matrix Visualization
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Human', 'AI'], 
            yticklabels=['Human', 'AI'],
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Test Set', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate additional metrics from confusion matrix
tn, fp, fn, tp = cm.ravel()
print("\nConfusion Matrix Breakdown:")
print(f"  True Negatives (Human → Human):  {tn}")
print(f"  False Positives (Human → AI):    {fp}")
print(f"  False Negatives (AI → Human):    {fn}")
print(f"  True Positives (AI → AI):        {tp}")
print(f"\nSpecificity (True Negative Rate): {tn/(tn+fp):.4f}")
print(f"Sensitivity (True Positive Rate): {tp/(tp+fn):.4f}")

In [ ]:
# ROC Curve
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_true, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Test Set', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 9. Analysis & Interpretation

### What Semantic Information Do Averaged Embeddings Capture?

Averaged word embeddings (like GloVe) capture **distributional semantic information** at the word level. Each word is represented as a vector in a high-dimensional space where words with similar meanings or contexts are positioned closer together. When we average these vectors across all words in a paragraph, we obtain:

1. **Topic/Theme Information**: The averaged vector represents the overall semantic "center of mass" of the paragraph, capturing the dominant topics and themes discussed.

2. **Lexical Composition**: The vocabulary choices are preserved—formal vs. informal language, technical vs. common terminology, abstract vs. concrete concepts.

3. **Semantic Similarity**: Paragraphs with similar meanings will have similar averaged vectors, regardless of exact word order or syntax.

4. **Word-level Patterns**: Common word associations and collocations are implicitly captured since pre-trained embeddings encode co-occurrence statistics from large corpora.

**However, averaged embeddings lose:**
- **Word order information**: "The dog bit the man" vs. "The man bit the dog" would have identical representations.
- **Syntactic structure**: Complex grammatical patterns, sentence structure, and dependencies are completely discarded.
- **Context-dependent meanings**: Polysemous words (e.g., "bank" as financial institution vs. river bank) receive the same embedding regardless of context.
- **Long-range dependencies**: Relationships between distant words in the text are not modeled.

---

### Why This Model May Perform Better Than Surface Statistics But Worse Than Transformers

**Better than Surface Statistics (Tier A):**

Surface-level features like average word length, punctuation density, or simple n-gram frequencies can be easily mimicked by modern language models. Averaged embeddings go deeper by:

- **Semantic Understanding**: Capturing meaning rather than just form. Two texts can use different words but similar concepts, and embeddings will recognize this similarity.
- **Vocabulary Richness**: Distinguishing between rich, varied vocabulary (typical of human literature) vs. repetitive or limited word choices (potential AI characteristic).
- **Generalization**: Pre-trained embeddings generalize across domains because they're trained on massive corpora, unlike simple statistical features that may be dataset-specific.

**Worse than Transformers (Tier C):**

Transformer-based models (BERT, GPT, RoBERTa) vastly outperform averaged embeddings because they:

1. **Context-Aware Representations**: Transformers generate contextual embeddings where each word's representation depends on the surrounding words, capturing nuanced meanings.

2. **Sequential Modeling**: Self-attention mechanisms in transformers model complex relationships between all words in the text, preserving word order and long-range dependencies.

3. **Hierarchical Features**: Transformers learn multiple layers of abstraction, from syntactic patterns to high-level semantic understanding, through their deep architecture.

4. **Fine-tuning Capability**: Transformers can be fine-tuned on specific tasks, adapting their representations to the nuances of AI vs. human text detection.

5. **Stylistic Patterns**: Transformers can detect subtle stylistic signatures like sentence rhythm, coherence patterns, and discourse structure that averaged embeddings completely miss.

---

### Strengths and Limitations of Averaged Embeddings for AI-Text Detection

**Strengths:**

1. **Simplicity & Interpretability**: The model is straightforward to understand and debug. We can examine which words contribute most to the classification.

2. **Efficiency**: Training is fast and requires minimal computational resources compared to transformers. Inference is nearly instantaneous.

3. **Low Data Requirements**: Feedforward networks with averaged embeddings can work reasonably well even with limited training data (thousands vs. millions of examples).

4. **No Pre-training Needed**: We leverage pre-trained GloVe embeddings without additional fine-tuning, making the approach accessible and reproducible.

5. **Semantic Baseline**: Provides a strong baseline that captures meaningful semantic differences between AI and human text beyond superficial statistics.

**Limitations:**

1. **Loss of Syntax**: Cannot detect grammatical errors, unnatural sentence structures, or syntactic patterns that might distinguish AI text.

2. **Insensitivity to Order**: "The enemy destroyed the fortress" and "The fortress destroyed the enemy" are indistinguishable—a critical flaw for coherence detection.

3. **Fixed Representations**: Word embeddings are static; the same word always has the same representation regardless of context.

4. **Averaging Artifacts**: Important words (like negations "not") can be drowned out when averaging hundreds of words, potentially reversing the intended meaning.

5. **Brittleness to Adversarial Attacks**: AI text generators could easily evade detection by simply adjusting vocabulary distributions to mimic human word choices while maintaining their characteristic syntactic patterns.

6. **Domain Dependence**: Performance may degrade when applied to domains different from both the GloVe training corpus and our novel-based dataset.

7. **No Stylistic Modeling**: Cannot capture coherence, logical flow, rhetorical devices, or the "human touch" in creative writing that transformers might detect through attention patterns.

**Conclusion:**

Averaged embeddings with a feedforward neural network represent a reasonable middle ground for AI text detection—better than purely surface-level features but fundamentally limited by the loss of sequential and contextual information. This approach serves as a strong baseline and is useful for scenarios where computational resources are constrained or interpretability is prioritized over maximum accuracy.